In [1]:
#include <iostream>
#include <stdlib.h>
#include <fstream>
#include <cstring>
#include <cctype>
#include <string>
using namespace std;
const char *cmd[] = { "sim", "5", "59", "tests/t01.in", "./P5_results.txt" };

In [2]:
#pragma cling add_include_path("~/OneDrive/Notebooks/CSCI 211/Projects/p5")
#pragma cling load("cust.h")
#pragma cling load("cust.cpp")
#pragma cling load("pqueue.h")
#pragma cling load("pqueue.cpp")

In [3]:
struct Checker{
    int m_money;
    int m_done_time;
    int m_num;
    Cust *m_curr_cust;
    bool m_been_robbed;
    bool m_unused;
};

In [4]:
//Array wrapper
//Allows for dynamic arrays to be traversed by range-based
//for loops
template <typename T>
struct wrapped_array {
    wrapped_array(T* first, T* last) : begin_ {first}, end_ {last} {}
    wrapped_array(T* first, ptrdiff_t size)
        : wrapped_array {first, first + size} {}

    T*  begin() const noexcept { return begin_; }
    T*  end() const noexcept { return end_; }

    T* begin_;
    T* end_;
};
template <typename T>
//wrapped_array is the same type as the array it contains
//returns pointer to first element and size of array
wrapped_array<T> wrap_array(T* first, ptrdiff_t size) noexcept
{ return {first, size}; }

In [5]:
//loads appropriate Pqueues; called by run_simulation()
void load_queues(Pqueue &arrival_queue, Pqueue &shopping_queue, Pqueue &checkout_queue, int &clock, ostream &os){
    //load shopping queue; dequeue arrival queue
    while(arrival_queue.first_priority() == clock){
        Cust *cust = arrival_queue.dequeue();
        cust->print_entered(os, clock);
        shopping_queue.enqueue(cust, (cust->get_priority() + (cust->get_items()*2)));
    }
    //dequeue shopping queue and load checkout queue
    while(shopping_queue.first_priority() == clock){
        Cust *cust = shopping_queue.dequeue();
        cust->print_done_shopping(os, clock);
        checkout_queue.enqueue(cust, 0);
    }
}

In [6]:
//performs appropriate actions upon a "robber is checking out" condition; called by done_checkout_handler()
void robber_checkout_helper(Checker &current, const int &break_duration, ostream &os, int &clock, int &num_customers){
    int cash = current.m_money;
    current.m_money = 0;
    current.m_been_robbed = true;
    //update done time after robbery for cashier
    current.m_done_time = clock + break_duration;
    current.m_curr_cust->print_done_checkout(os, clock, current.m_num, cash);
    Cust *ptr = current.m_curr_cust;
    current.m_curr_cust = nullptr;
    //delete customer after done checkout
    delete ptr;
    //update total number of customers
    num_customers--;
}

In [7]:
//same as robber_checkout_helper() except for "shopper is checking out"; called by done_checkout_handler()
void shopper_checkout_helper(Checker &current, ostream &os, int &clock, int &num_customers){
    //all items are $3
    int cash = (current.m_curr_cust->get_items())*3;
    //update cashier cash after customer buys items
    current.m_money += cash;
    current.m_curr_cust->print_done_checkout(os, clock, current.m_num, cash);
    Cust *ptr = current.m_curr_cust;
    current.m_curr_cust = nullptr;
    delete ptr;
    num_customers--;
}

In [8]:
//calls helper functions depending if shopper or robber is at checkout and prints respective messages; called by run_simulation()
void done_checkout_handler(const int &num_checkers, Checker* &checker_array, int &clock, const int &break_duration, int &num_customers, ostream &os){
    for(auto&& index : wrap_array(checker_array, num_checkers)){
        if(index.m_done_time == clock){
            //decides if robber is at checkout or shopper is at checkout and calls respective functions
            if(index.m_curr_cust != nullptr && index.m_curr_cust->is_robber()){
                robber_checkout_helper(index, break_duration, os, clock, num_customers);
            }else if(index.m_curr_cust != nullptr){
                shopper_checkout_helper(index, os, clock, num_customers);
            }
        }
    }
}

In [9]:
//updates Checker clock member variable and prints started-checkout message; called by run_simulation()
void start_checkout_handler(const int &num_checkers, Checker* &checker_array, Pqueue &checkout_queue, int &clock, ostream &os){
    for(auto&& index : wrap_array(checker_array, num_checkers)){
        if(index.m_curr_cust == nullptr && !checkout_queue.empty() && index.m_done_time <= clock){
            Cust *cust = checkout_queue.dequeue();
            index.m_curr_cust = cust;
            index.m_unused = false;
            if(cust->is_robber()){
                //robberies take 7 clock ticks no matter the amount of items
                index.m_done_time = clock + 7;
            }else{
                //purchasing time is dependent on number of items
                index.m_done_time = clock + cust->get_items();
            }
            cust->print_started_checkout(os, clock, index.m_num);
        }
    }
}

In [10]:
void run_simulation(Pqueue &arrival_queue, const int num_checkers, const int break_duration, ostream &os){
    Pqueue shopping_queue, checkout_queue;
    int clock, num_customers = arrival_queue.length();
    //create an array of Checker structs
    Checker *checker_array = (Checker*) calloc(num_checkers, sizeof(Checker));
    for(int index = 0; index < num_checkers; index++){
        checker_array[index] = { 250, 0, index, nullptr, false, true };
    }
    os<<"*STORE OPEN*"<<endl<<endl;
    for(clock = 1; num_customers > 0; clock++){
        load_queues(arrival_queue, shopping_queue, checkout_queue, clock, os);
        done_checkout_handler(num_checkers, checker_array, clock, break_duration, num_customers, os);
        start_checkout_handler(num_checkers, checker_array, checkout_queue, clock, os);
    }
    os<<endl<<"*STORE CLOSED*"<<endl<<endl;
    for(auto&& index : wrap_array(checker_array, num_checkers)){
        if(index.m_been_robbed){
            os<<"Register <"<<(index.m_num+1)<<"> = $"<<index.m_money<<endl<<"\tThis register was robbed during sim!"<<endl<<endl;
        }else if(index.m_unused){
            os<<"Register <"<<(index.m_num+1)<<"> = $"<<index.m_money<<endl<<"\tThis register was never used during sim."<<endl<<endl;
        }else{
            os<<"Register <"<<(index.m_num+1)<<"> = $"<<index.m_money<<endl<<endl;
        }
    }
    os<<"End Time = "<<clock<<"."<<endl<<endl;
    free(checker_array);
}

In [11]:
// Return true if str is a number, false otherwise.
bool legal_int_cmd(const char *str){
    bool result = false;
    // Check for null pointer.
    if(str != NULL && strlen(str) > 0){
        int i, len = strlen(str);
        // Single character case.
        if(len == 1){
            result = isdigit(str[0]);
        }else{
            // Check that every character is a digit.
            try{
                for(i = 0; i < len; i++){
                    if(!isdigit(str[i])){
                        throw false;
                    }
                }
                result = true;
            }catch(bool error){
                result = error;
            }catch(...){
                throw;
            }
        }
    }
    return result;
}

In [12]:
//Check for errors in commandline arguments
//Error checking ordered as per instructions request 
void read_commands(int argc, const char *argv[], ifstream &ifile, ofstream &ofile, string &error){
    if(argc != 5){
        throw "Error: invalid number of command line arguments.";
    }
    if(!ifile.is_open()){
        error = "Error: could not open input file <";
        error += argv[3];
        error += ">.";
        throw error.c_str();
    }
    if(!ofile.is_open()){
        error = "Error: could not open output file <";
        error += argv[4];
        error += ">.";
        throw error.c_str();
    }
    if(!(legal_int_cmd(*(argv + 1))) || atoi(argv[1]) < 1){
        throw "Error: invalid number of checkers specified.";
    }
    if(!(legal_int_cmd(*(argv + 2))) || atoi(argv[2]) < 0){
        throw "Error: invalid checker break duration specified.";
    }
}

In [13]:
void read_input(Pqueue &arrival_queue, ifstream &ifile){
    string name, type;
    int time, items;
    while(!ifile.eof()){
        ifile>>ws>>name>>type>>time>>items>>ws;
        arrival_queue.enqueue(new Cust(name, (type == "robber" ? true : false), time, items), time);
    }
}

In [14]:
void runner(const char* argv[]){
    int result = 0;
    int argc = 5;
    string err;
    //const char *argv[] = { "sim", "3", "0", "tests/t01.in", "./P5_results.txt" };
    try{        
        Pqueue arrival_queue;
        ifstream ifile(argv[3], ios::in);
        ofstream ofile(argv[4], ios::out);
        //call function to error check cmdline arguments
        read_commands(argc, argv, ifile, ofile, err);
        
        //read input file until EOF
        read_input(arrival_queue, ifile);
        
        //pass arrival queue and start simulation
        run_simulation(arrival_queue, atoi(argv[1]), atoi(argv[2]), ofile);
        ifile.close();
        ofile.close();
    }catch(const char* error){
        cerr << error << endl;
        result = 1;
    }
    if(result==0)
        cout << "Process Complete!" << endl;
}

#### main()

In [15]:
runner(cmd);

Process Complete!


# Statistics

In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t01.in with 1 register

924.183 us +- 79.7678 us per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t01.in with 5 registers

1.06966 ms +- 145.161 us per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t14.in with ~10 registers

3.22742 ms +- 180.859 us per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 148 registers and 10000 break time

401.068 ms +- 5.51678 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 69 registers

401.449 ms +- 6.36622 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 148 registers and 1000 break time

409.98 ms +- 4.22518 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 420 registers and 1000 break time

410.578 ms +- 5.21259 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 100 registers and 1000 break time

427.444 ms +- 5.09663 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 69 registers and 1000 break time

498.744 ms +- 4.76103 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 69 registers and 10000 break time

512.517 ms +- 9.0743 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


In [15]:
%timeit -n 1 -r 100 -p 6 runner(cmd); //t15.in with 35 registers and 10000 break time

676.896 ms +- 6.11763 ms per loop (mean +- std. dev. of 100 runs 1 loop each)


# OUT

In [16]:
ifstream output("./P5_results.txt");
if(output.is_open())
    cout << output.rdbuf();

*STORE OPEN*

2: Lisa entered store
3: Homer entered store
5: Bart entered store
21: Homer done shopping
21: Homer started checkout with checker 0
26: Lisa done shopping
26: Lisa started checkout with checker 1
27: Maggie entered store
29: Maggie done shopping
29: Maggie started checkout with checker 2
30: Homer paid $27 for 9 items to checker 0
36: Maggie stole $250 and 1 item from checker 2
38: Lisa paid $36 for 12 items to checker 1
51: Bart done shopping
51: Bart started checkout with checker 0
74: Bart paid $69 for 23 items to checker 0

*STORE CLOSED*

Register <1> = $346

Register <2> = $286

Register <3> = $0
	This register was robbed during sim!

Register <4> = $250
	This register was never used during sim.

Register <5> = $250
	This register was never used during sim.

End Time = 75.



# Test Zone

# Archive

In [ ]:
void done_checkout_handler(const int &num_checkers, Checker* &checker_array, int &clock, const int &break_duration, int &num_customers, ostream &os){
    for(int index = 0; index < num_checkers; index++){
        if(checker_array[index].m_done_time == clock){
            if(checker_array[index].m_curr_cust != nullptr && checker_array[index].m_curr_cust->is_robber()){
                robber_checkout_helper(checker_array, index, break_duration, os, clock, num_customers);
            }else if(checker_array[index].m_curr_cust != nullptr){
                shopper_checkout_helper(checker_array, index, os, clock, num_customers);
            }
        }
    }
}

In [ ]:
void start_checkout_handler(const int &num_checkers, Checker* &checker_array, Pqueue &checkout_queue, int &clock, ostream &os){
    for(int index = 0; index < num_checkers; index++){
        if(checker_array[index].m_curr_cust == nullptr && !checkout_queue.empty() && checker_array[index].m_done_time <= clock){
            Cust *cust = checkout_queue.dequeue();
            checker_array[index].m_curr_cust = cust;
            checker_array[index].m_unused = false;
            if(cust->is_robber()){
                checker_array[index].m_done_time = clock + 7;
            }else{
                checker_array[index].m_done_time = clock + cust->get_items();
            }
            cust->print_started_checkout(os, clock, index);
        }
    }
}

In [ ]:
void run_simulation(Pqueue &arrival_queue, const int num_checkers, const int break_duration, ostream &os){
    Pqueue shopping_queue, checkout_queue;
    int clock, num_customers = arrival_queue.length();
    //create an array of Checker structs
    Checker *checker_array = (Checker*) calloc(num_checkers, sizeof(Checker));
    for(int index = 0; index < num_checkers; index++){
        checker_array[index] = { 250, 0, index, nullptr, false, true };
    }
    for(clock = 1; num_customers > 0; clock++){
        load_shopping_queue(arrival_queue, shopping_queue, clock, os);
        load_checkout_queue(shopping_queue, checkout_queue, clock, os);
        done_checkout_handler(num_checkers, checker_array, clock, break_duration, num_customers, os);
        start_checkout_handler(num_checkers, checker_array, checkout_queue, clock, os);
    }
    for(int index = 0; index < num_checkers; index++){
        if(checker_array[index].m_been_robbed){
            os<<"registers["<<index<<"] = $"<<checker_array[index].m_money<<" was robbed during sim!"<<endl;
        }else if(checker_array[index].m_unused){
            os<<"registers["<<index<<"] = $"<<checker_array[index].m_money<<" was never used during sim."<<endl;
        }else{
            os<<"registers["<<index<<"] = $"<<checker_array[index].m_money<<endl;
        }
    }
    os<<"time = "<<clock<<endl;
    free(checker_array);
}